# Q1: Training ResNet-18 on CIFAR-100

**Objectives:**
- Train ResNet-18 classifier on CIFAR-100
- Visualize training curves
- Save checkpoint for later use
- Generate training GIF for report

## Setup

In [ ]:
# Mount Google Drive (for saving checkpoints)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone repository (if not already cloned)
import os
if not os.path.exists('/content/ood-neural-collapse'):
    !git clone https://github.com/DiegoFleury/OOD-Detection-Project---CSC_5IA23
%cd /content/ood-neural-collapse

In [ ]:
# Install dependencies
!pip install -q torch torchvision matplotlib seaborn scikit-learn pyyaml imageio tqdm

In [ ]:
# Imports
import torch
import numpy as np
import matplotlib.pyplot as plt
import yaml

from src.models import ResNet18
from src.data import get_cifar100_loaders
from src.utils import Trainer, plot_training_curves, create_training_gif, plot_final_metrics

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Load config
with open('configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration:")
print(yaml.dump(config, default_flow_style=False))

## 1. Load Data

In [ ]:
print("Loading CIFAR-100 dataset...")

train_loader, val_loader, test_loader = get_cifar100_loaders(
    data_dir=config['data']['data_dir'],
    batch_size=config['training']['batch_size'],
    num_workers=config['data']['num_workers'],
    augment=config['data']['augment'],
    val_split=config['training']['val_split']
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

In [ ]:
# Visualize sample batch
images, labels = next(iter(train_loader))

fig, axes = plt.subplots(2, 8, figsize=(16, 4))
for i, ax in enumerate(axes.flat):
    img = images[i].permute(1, 2, 0).numpy()
    # Denormalize
    img = img * np.array([0.2675, 0.2565, 0.2761]) + np.array([0.5071, 0.4867, 0.4408])
    img = np.clip(img, 0, 1)
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(f"Class {labels[i].item()}", fontsize=8)

plt.tight_layout()
plt.savefig('results/figures/training/sample_batch.png', dpi=150, bbox_inches='tight')
plt.show()

## 2. Create Model

In [ ]:
print("Creating ResNet-18 model...")

model = ResNet18(num_classes=config['model']['num_classes'])

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Test forward pass
x_test = torch.randn(2, 3, 32, 32)
with torch.no_grad():
    out = model(x_test)
print(f"Output shape: {out.shape}")

## 3. Train Model

In [ ]:
# Create trainer
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    lr=config['training']['lr'],
    momentum=config['training']['momentum'],
    weight_decay=config['training']['weight_decay'],
    device=device
)

print("Starting training...")
print("=" * 50)

In [ ]:
# Train the model

history = trainer.train(
    epochs=config['training']['epochs'],
    save_dir=config['paths']['checkpoints'],
    early_stopping_patience=config['training']['early_stopping_patience']
)

## 4. Visualize Results

In [ ]:
# Plot training curves
plot_training_curves(
    history,
    save_path='results/figures/training/training_curves.png'
)

In [ ]:
# Plot final metrics summary
plot_final_metrics(
    history,
    save_path='results/figures/training/final_metrics.png'
)

In [ ]:
# Create animated GIF (for README)
print("Creating training GIF...")
create_training_gif(
    history,
    save_path='results/figures/gifs/training_curves.gif',
    fps=10
)
print("GIF saved!")

## 5. Save Final Checkpoint to Google Drive

In [ ]:
import shutil

drive_checkpoint_dir = '/content/drive/MyDrive/ood-neural-collapse/checkpoints'
os.makedirs(drive_checkpoint_dir, exist_ok=True)

shutil.copy(
    'checkpoints/resnet18_cifar100_best.pth',
    os.path.join(drive_checkpoint_dir, 'resnet18_cifar100_best.pth')
)

print(f"✓ Checkpoint saved to Google Drive: {drive_checkpoint_dir}")

## 6. Print Final Summary

In [ ]:
print("\n" + "=" * 60)
print("TRAINING SUMMARY")
print("=" * 60)

print(f"\nFinal Train Accuracy: {history['train_acc'][-1]:.2f}%")
print(f"Final Val Accuracy: {history['val_acc'][-1]:.2f}%")
print(f"Final Test Accuracy: {history['test_acc'][-1]:.2f}%")

best_val_acc = max(history['val_acc'])
best_epoch = history['val_acc'].index(best_val_acc) + 1
print(f"\nBest Val Accuracy: {best_val_acc:.2f}% (Epoch {best_epoch})")

print(f"\nTotal Epochs: {len(history['train_loss'])}")
print(f"\nCheckpoint saved at: checkpoints/resnet18_cifar100_best.pth")
print(f"Figures saved in: results/figures/training/")
print("\n" + "=" * 60)

## 7. Commit Results to GitHub

In [ ]:
!git add results/figures/training/
!git add results/figures/gifs/training_curves.gif
!git commit -m "Add Q1 training results: curves and GIF"
!git push

print("Results committed to GitHub!")